# Analyse av enzymkinetikkdata

I forrige laboratorieoppgave gjorde dere kinetikkmålinger på enzymet alkalisk fosfatase, som er vist nedenfor.

In [6]:
import py3Dmol

d = py3Dmol.view(query="pdb:1ANJ")
d.setStyle({"cartoon": {"color": "spectrum"}})
d.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Du skal nå lære hvordan du kan visualisere og analysere enzymkinetikkdata med Python. Du kan bruke en valgfri editor til å lage programmene som gjør analysen av data fra enzymkinetikklaben, for eksempel Jupyter Notebook. Figurene og resultatene skal brukes i labrapporten. Hvis du står fast, kan du spørre generativ kunstig intelligens (gpt.uio.no) om hjelp. Da kan det være lurt å spesifisere hvilken del av koden du trenger hjelp til ved å lime inn koden og formatet på datafilene dine. Bruk GPT4o som modell. Hvis dette heller ikke hjelper, kan du spørre labansvarlige om hjelp (veiledningstime uka etter laben).

## Steg 1: Finne initialhastighetene

Før vi kan begynne å analysere en Michaelis-Menten-modell for dataene våre, må vi finne initialhastigheten til reaksjonen som en funksjon av substratkonsentrasjonen fra dataene våre av absorbansøkning over tid.

### Importere data
Du har lagret data fra enzymkinetikklaben som separate csv-filer. Fila inneholder to kolonner: tid og absorbans. Vi skal bruke disse dataene til å finne initialhastigheten til reaksjonen som en funksjon av substratkonsentrasjonen.

```{code-block} Python

La oss se på hvordan vi kan lese av fila. Det finnes mange måter å importere filer på i Python. Du kan velge den metoden du er mest komfortabel med. Det enkleste metoden er å bruke _read\_csv_ fra pandas-biblioteket:

```{code-block} Python
import pandas as pd

data = pd.read_csv("Absorbans_tid.csv") # Leser fila

tid = data["tid"]
absorbans = data["absorbans"]
```

Fila du skal lese av, må ligge i samme mappe som programmet som skal analysere og lese fila. Eventuelt kan vi angi filbanen, f.eks. "datafiler/Absorbans_tid.csv". Vi kan også lese direkte fra internett. Da erstatter du filnavnet med en lenke.

Du skal lese fem filer, en for hver substratkonsentrasjon. Du kan bruke en løkke for å lese alle filene. 

````{admonition} Oppgave
:class: tip
Lag et program som leser alle filene og lagrer dataene i lister. Du kan ta utgangspunkt i programmet nedenfor. Prøv selv før du kikker på løsningsforslaget!

```{code-block} Python  
import pandas as pd
import matplotlib.pyplot as plt

# Liste over filnavn
data_files = [f"data{i}.csv" for i in range(1, 6)] # Denne lista kan også lages manuelt (["data1.csv", "data2.csv", ...])

# Les og plott data fra alle filer med en løkke som går over alle filnavnene
# Fyll inn koden her

plt.xlabel("Tid (min)")
plt.ylabel("Absorbans")
plt.legend()
plt.grid()
plt.show()
```
````

````{admonition} Løsningsforslag
:class: tip,dropdown
```{code-block} Python
import pandas as pd
import matplotlib.pyplot as plt

# Liste over filnavn
data_files = [f"data{i}.csv" for i in range(1, 6)] # Denne lista kan også lages manuelt (["data1.csv", "data2.csv", ...])

# Les og plott data fra alle filer
for file in data_files:
    df = pd.read_csv(file, delimiter=";")
    t = df["tid_min"]
    A = df["abs"]
    plt.scatter(t, A, label=file)

plt.xlabel("Tid (min)")
plt.ylabel("Absorbans")
plt.title("Absorbansmålinger uten hemmer")
plt.legend()
plt.grid()
plt.show()
```
````

### Lineærtilpasning av dataene

Lineær regresjon betyr å finne et polynom av første grad (en førstegradsfunksjon) som gir minst mulig gjennomsnittlig varians til et sett med datapunkter. Metodene vi skal bruke, benytter minste kvadraters metode for å estimere en slik funksjon. En funksjon som er ganske enkel å bruke til dette, er _polyfit_ fra numpy-biblioteket. Du kjenner kanskje til og med denne funksjonen fra før (f.eks. fra IN-KJM1900). Denne funksjonen utfører polynomregresjon med en vilkårlig grad av polynomet. 

Det polyfit-funksjonen derimot ikke kan estimere direkte, er andre typer regresjonsmodeller enn lineære, slik som eksponentialfunksjoner, logaritmefunksjoner eller logistiske funksjoner. Vi skal gjøre en logistisk regresjon seinere, og derfor benytter vi heller en annen funksjon som heter _curve\_fit_ fra scipy-biblioteket også når vi skal gjøre lineær regresjon i denne laben. Denne funksjonen returnerer både koeffisientene i regresjonslinja og et mål på hvor godt modellen passer dataene (såkalt kovarians). Det er koeffisientene _a_ og _b_ i uttrykket $f(x) = ax + b$ som estimeres når vi gjør en linær regresjon.

I _curve\_fit_ må vi definere en funksjonsmodell på formen $f(x, a, b, c, ...)$ som brukes som utgangspunkt for regresjonen. Her skal vi definere en funksjon $f(x, a, b) = ax + b$.

Nå skal du gjøre en oppgavene/nivåene nedenfor. Velg kun én av nivåene. Dersom du mener du behersker enkel plotting og regresjon med curve\_fit, velger du nivå 1. Dersom du er usikker på hvordan dette gjøres, kan du velge nivå 2. Prøv gjerne på nivå 1 før du prøver på nivå 2 - da lærer du enda mer! Du kan laste ned filene fra Canvas og bruke dem i en valgfri editor, som Jupyter Notebook eller Spyder.

```{admonition} Oppgave
:class: tip
1. Se på programmmet nedenfor og forklar hva det gjør, trinn for trinn.
2. Hva beskriver variabelen _koeff_?
3. Hvordan kan vi plotte originaldataene sammen med regresjonslinja?
```

```{code-block} Python
import pandas as pd
import numpy as np
import scipy.optimize as opt

data = pd.read_csv("Absorbans_tid.csv")

tid = data["tid"]
absorbans = data["absorbans"]

# Definerer formen på modellen vi ønsker
def modell(x, a, b):
    return a*x + b

koeff, covar = opt.curve_fit(modell, tid, absorbans)
# Finner a og b her
a = koeff[0]
b = koeff[1]

# Beregner funksjonsverdier til den lineære tilpasningen
x = np.linspace(0,35,100) # Nye x-verdier å plotte med
y = a*x + b               # Regresjonsmodellens y-verdier
````
`````

### Håndtering av flere datafiler
Vi skal nå kombinere de to stegene ovenfor for å analysere alle dataene våre. Vi skal gjøre en lineær regresjon for hver fil og plotte regresjonslinjene sammen med originaldataene.

```{admonition} Oppgave
:class: tip
Prøv å kombinere løkka som leser alle dataene med regresjonen av dataene. Du kan ta utgangspunkt i programmene ovenfor. Prøv selv før du kikker på løsningsforslaget!
```

````{admonition} Løsningsforslag
:class: tip,dropdown

```{code-block} Python
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

# Liste over filnavn
data_files = [f"data{i}.csv" for i in range(1, 6)] # Denne lista kan også lages manuelt (["data1.csv", "data2.csv", ...])
S = [0.257, 0.0634, 0.0343, 0.0171, 0.0114] # Konsentrasjoner (mg/mL). 
# Definerer formen på modellen vi ønsker
def modell(x, a, b):
    return a*x + b

V0 = [] # Initialhastigheter

# Les og plott data fra alle filer
for file, kons in zip(data_files, konsentrasjoner): # kan også bruke for fole in data_files: dersom du ikke har med navnelapper på konsentrasjonene
    df = pd.read_csv(file, delimiter=";")
    t = df["tid_min"]
    A = df["abs"]

    koeff, covar = opt.curve_fit(modell, t, A)
    a = koeff[0]
    b = koeff[1]
    
    # Beregner funksjonsverdier til den lineære tilpasningen
    x = np.linspace(0,15) # Nye x-verdier å plotte med
    y = a*x + b               # Regresjonsmodellens y-verdier

    v0.append(a) # Legger til initialhastigheten i lista
    plt.scatter(t, A, label=str(kons) + " mg/mL")
    plt.plot(x, y, label=f"y = {a:.2f}x + {b:.2f}")

plt.xlabel("Tid (min)")
plt.ylabel("Absorbans")
plt.title("Absorbansmålinger av alkalisk fosfatase med PNPP")
plt.legend()
plt.grid()
plt.show()
```
````
3. Lagre grafen som et høyoppløselig bilde, og lagre dataene dine  i en ny csv-fil. Dette kan du gjøre ved å legge til følgende kode nederst i programmet ditt (men før plt.show()):

In [ ]:
plt.savefig("initialhastigheter.tiff") # kan også angi oppløsning med f.eks. dpi=500 (500 piksler per tomme)

# Lagrer dataene i en dataframe og skriver dem til en csv-fil
d = {"Hastighet": V0, "Substratkonsentrasjon": S}
df = pd.DataFrame(d)
df.to_csv("Reaksjonshastigheter.csv", index = False)

## Steg 2: Finne vmaks og Km

### Michaelis-Menten-modellen
Vi skal nå plotte og analysere resultatene fra første analyse. Vi skal altså lage et Michaelis-Menten-plott og tilpasse Michaelis-Menten-likningen til verdiene vi fant for initialhastighetene. Vi har lagret substratkonsentrasjonene og de tilhørende initialhastighetene i en .csv-fil (reaksjonshastigheter.csv), og vi skal bruke denne fila videre.

```{admonition} Oppgave
:class: tip
1. Les av fila som du lagde i forrige program, og skriv ut initialhastighetene og tilhørende substratkonsentrasjoner for å se at du har lest inn fila korrekt.
2. Plott dataene (gjerne med scatter, slik at du får punkter istedenfor linjer) slik at du får et Michaelis-Menten-plott (husk hvilken form grafen skal ha!). Husk aksetitler med korrekte enheter.
```

Nå skal vi lage en modellkurve som er basert på Michaelis-Menten-likningen. Denne kan vi definere som en Python-funksjon, slik (på samme måte som du definerte den lineære funksjonen i del 1):

In [7]:
def michaelis_menten(x, Vmax, Km):
    return Vmax*x/(Km + x)

```{admonition} Oppgave
:class: tip
1. Bruk funksjonen ovenfor og gjør en regresjon av initialhastighetene med _curve\_fit_ på samme måte som med den lineære funksjonen (men nå skal du bruke michaelis-menten-funksjonen som regresjonsmodell isteden).
2. Plott regresjonskurven i samme koordinatsystem som dataene dine.
3. Modifiser programmet slik at Km får enheten mol/L (M) og Vmax får enheten mol/L/min (M/min) (dette er ikke gjort i løsningsforslaget, så det må du gjøre selv). Ekstinksjonskoeffisienten, ε, til produktet er 18 000 M/cm. Lysveien, l (lengden av kyvetten), er 1 cm.
```

```{admonition} Løsningsforslag
:class: tip,dropdown
```{code-block} Python
import pandas as pd
import numpy as np

data = pd.read_csv("reaksjonshastigheter.csv")

S = data["Substratkonsentrasjon"]
V0 = data["Initialhastighet"]

def michaelis_menten(S, Vmax, Km):
    return Vmax*S/(Km + S)

koeff, covar = opt.curve_fit(michaelis_menten, S, V0)
Vmax = koeff[0]
Km = koeff[1]

# Beregner funksjonsverdier til Michaelis-Menten-tilpasningen
S = np.linspace(0,0.3,100) # Nye x-verdier å plotte med
V = michaelis_menten(S, Vmax, Km) # Regresjonsmodellens y-verdier

plt.scatter(S, V0, label="Data")
plt.plot(S, V, label=f"Michaelis-Menten: Vmax = {Vmax:.2f}, Km = {Km:.2f}")

plt.xlabel("Substratkonsentrasjon (mg/mL)")
plt.ylabel("Initialhastighet (mg/mL/min)")
plt.title("Michaelis-Menten-plot")
plt.legend()
plt.grid()
plt.show()
```
```

### Enzymhemming
For å avgjøre om en hemmer er ikke-konkurrerende eller konkurrerende, kan vi plotte en modell for initalhastigheter med og uten hemmer i samme koordinatsystem. Du kan bruke samme framgangsmåte som i del 1, der du lagde en løkke som gikk igjennom flere filer med data, analyserte og plottet dataene. Vi skal bruke et Michaelis-Menten-plott for å sammenlikne resultatene for et enzym med og uten hemmer. Dersom du vil, kan du også prøve å lage et Lineweaver-Burk-plott.

```{admonition} Oppgave
:class: tip
1. Bruk programmet du lagde i del 1 for å lage en fil med initialhastigheter med hemmer.
2. Bruk programmet du lagde i del 2 for å konstruere det samme MM-plottet du lagde ovenfor, og legg til et MM-plott for dataene med hemmer i samme koordinatsystem.
3. Kjør programmet og forklar formen på grafen. Hva slags type hemmer har vi å gjøre med her?
```

````{admonition} Løsningsforslag
Løsningsforslag på oppgave 2. I oppgave 1 kan du mer eller mindre bruke det samme programmet som i del 1 - bare juster for filnavn og konsentrasjoner.
:class: tip,dropdown
```{code-block} Python
import pandas as pd
import numpy as np

data = pd.read_csv("reaksjonshastigheter.csv")

S = data["Substratkonsentrasjon"]
V0 = data["Initialhastighet"]

def michaelis_menten(S, Vmax, Km):
    return Vmax*S/(Km + S)

koeff, covar = opt.curve_fit(michaelis_menten, S, V0)
Vmax = koeff[0]
Km = koeff[1]

# Beregner funksjonsverdier til Michaelis-Menten-tilpasningen
S = np.linspace(0,0.3,100) # Nye x-verdier å plotte med
V = michaelis_menten(S, Vmax, Km) # Regresjonsmodellens y-verdier

plt.scatter(S, V0, label="Data")
plt.plot(S, V, label=f"Michaelis-Menten: Vmax = {Vmax:.2f}, Km = {Km:.2f}")

plt.xlabel("Substratkonsentrasjon (mg/mL)")
plt.ylabel("Initialhastighet (mg/mL/min)")
plt.title("Michaelis-Menten-plot")
plt.legend()
plt.grid()

# Legg til data med hemmer
data_hemmer = pd.read_csv("reaksjonshastigheter_hemmer.csv")

S_hemmer = data_hemmer["Substratkonsentrasjon"]
V0_hemmer = data_hemmer["Initialhastighet"]

koeff_hemmer, covar_hemmer = opt.curve_fit(michaelis_menten, S_hemmer, V0_hemmer)
Vmax_hemmer = koeff_hemmer[0]
Km_hemmer = koeff_hemmer[1]

V_hemmer = michaelis_menten(S, Vmax_hemmer, Km_hemmer) # Regresjonsmodellens y-verdier

plt.scatter(S_hemmer, V0_hemmer, label="Data med hemmer")
plt.plot(S, V_hemmer, label=f"Michaelis-Menten med hemmer: Vmax = {Vmax_hemmer:.2f}, Km = {Km_hemmer:.2f}")

plt.show()
```
````

```{admonition} Oppgave
:class: tip
Lagre alle figurer du trenger med _savefig_-kommandoen og print ut eller lagre alle koeffisientene du har funnet. Bruk gjerne en tabell for å presentere resultatene. Skriv labrapporten din basert på resultatene du har funnet. Du kan bruke malen for labrapporten som ligger på Canvas.
```